<a href="https://colab.research.google.com/github/hurshd0/DS-Unit-2-Regression-Classification/blob/master/module3/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`) using a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do exploratory visualizations with Seaborn.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [X] Fit a linear regression model with multiple features.
- [X] Get mean absolute error for the test set.
- [X] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [X] Add your own stretch goal(s) !
- [X] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).
(That book is good regardless of whether your cultural worldview is inferential statistics or predictive machine learning)
- [ ] Read Leo Breiman's paper, ["Statistical Modeling: The Two Cultures"](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/hurshd0/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module3')

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/hurshd0/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a 
# future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')


################################ EDA IMPORTS ###################################
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt # plotting lib
import seaborn as sns # matplotlib wrapper plotting lib
import plotly.graph_objs as go # interactive low-level plotting lib https://plot.ly/python/
import plotly.express as px #high-level api wrapper for plotly https://plot.ly/python/plotly-express/#visualize-distributions

# ---------------- Plot libs settings ------------- #
plt.style.use('seaborn-darkgrid')
sns.set(context='notebook', style='darkgrid', palette='colorblind')

# ---------------- Pandas settings --------------- #
# Removes rows and columns truncation of '...'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
################################################################################

################################ STATS IMPORTS #################################

################################################################################


#################################### ML IMPORTS ################################
import category_encoders as ce
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, SelectKBest
################################################################################

In [0]:
#### HELPER FUNCTIONS ######

def correlations(data, y, xs):
    from scipy import stats
    rs = []
    rhos = []
    for x in xs:
        r = stats.pearsonr(data[y], data[x])[0]
        rs.append(r)
        rho = stats.spearmanr(data[y], data[x])[0]
        rhos.append(rho)
    return pd.DataFrame({"feature": xs, "r": rs, "rho": rhos})

def correlation_heatmap(data=None, vmax=1, annot=True, corr_type='pearson', figsize=(12, 12)):
    if data is None:
        raise ValueError(
            "The parameter 'data' must be assigned a non-nil reference to a Pandas DataFrame")
    # Taken from the seaborn example at:
    # http://seaborn.pydata.org/examples/many_pairwise_correlations.html
    # Compute the correlation matrix
    corr = data.corr(corr_type)
    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    # Set up the matplotlib figure
    fig, axes = plt.subplots(figsize=figsize)
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=vmax, annot=annot, square=True,
                linewidths=.5, cbar_kws={"shrink": .5}, fmt=',.2f', ax=axes)
    plt.show()
    plt.close()

# LOAD DATESET

In [4]:
def load_nyc_rolling_sales_data(filepath):
    # Read New York City property sales data
    df = pd.read_csv(filepath)

    # Change column names: replace spaces with underscores
    df.columns = [col.replace(' ', '_') for col in df]

    # SALE_PRICE was read as strings.
    # Remove symbols, convert to integer
    df['SALE_PRICE'] = (
        df['SALE_PRICE']
        .str.replace('$','')
        .str.replace('-','')
        .str.replace(',','')
        .astype(int)
    )

    # Keep subset of rows:
    # 1. BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS'
    # 2. SALE_PRICE > $100K AND SALE_PRICE < $2M

    mask = (
        (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
        ((df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000))
    )
    df = df[mask]

    # Drop unnecessary columns or columns that have > 90% NaNs
    df = df.drop(['ADDRESS', 'BUILDING_CLASS_CATEGORY','EASE-MENT','APARTMENT_NUMBER', 'TAX_CLASS_AT_PRESENT', 'TAX_CLASS_AT_TIME_OF_SALE'], axis=1)

    # Convert SALE_DATE, YEAR_BUILT to datetime
    df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
    df['YEAR_BUILT'] = pd.to_datetime(df['YEAR_BUILT'], infer_datetime_format=True).dt.year

    # Convert LAND_SQUARE_FEET to float
    df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',', '').astype(float)

    # Convert float to int
    float_cols = ['TOTAL_UNITS', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS']
    for col in float_cols:
        df[col] = df[col].astype(int)

    # Convert to categorical
    cat_cols = ['BOROUGH', 'TOTAL_UNITS', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS']
    for col in cat_cols:
        df[col] = df[col].astype(object)
    
    # Map BOROUGHS to their names
    boroughs_name = {1:'MANHATTAN', 2:'BROOKLYN', 3:'QUEENS', 4:'BRONX', 5:'STATEN ISLAND'}
    df['BOROUGH'] = df['BOROUGH'].map(boroughs_name)
    return df
    
raw_df = load_nyc_rolling_sales_data('../data/NYC_Citywide_Rolling_Calendar_Sales.csv')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3164 entries, 44 to 23035
Data columns (total 15 columns):
BOROUGH                           3164 non-null object
NEIGHBORHOOD                      3164 non-null object
BLOCK                             3164 non-null int64
LOT                               3164 non-null int64
BUILDING_CLASS_AT_PRESENT         3164 non-null object
ZIP_CODE                          3164 non-null float64
RESIDENTIAL_UNITS                 3164 non-null object
COMMERCIAL_UNITS                  3164 non-null object
TOTAL_UNITS                       3164 non-null object
LAND_SQUARE_FEET                  3164 non-null float64
GROSS_SQUARE_FEET                 3164 non-null float64
YEAR_BUILT                        3164 non-null int64
BUILDING_CLASS_AT_TIME_OF_SALE    3164 non-null object
SALE_PRICE                        3164 non-null int64
SALE_DATE                         3164 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(4), object(7)


In [0]:
# Get Pandas Profiling Report
# raw_df.profile_report()

In [6]:
raw_df.head()

,BOROUGH,NEIGHBORHOOD,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,QUEENS,OCEAN PARKWAY-NORTH,5495,801,A9,11230.0,1,0,1,6800.0,1325.0,1970,A9,550000,2019-01-01
61,BRONX,QUEENS VILLAGE,7918,72,A1,11427.0,1,0,1,4000.0,2001.0,1970,A1,200000,2019-01-01
78,BROOKLYN,PELHAM PARKWAY SOUTH,4210,19,A1,10461.0,1,0,1,3500.0,2043.0,1970,A1,810000,2019-01-02
108,QUEENS,FLATBUSH-CENTRAL,5212,69,A1,11226.0,1,0,1,4000.0,2680.0,1970,A1,125000,2019-01-02
111,QUEENS,FLATBUSH-EAST,7930,121,A5,11203.0,1,0,1,1710.0,1872.0,1970,A5,620000,2019-01-02


# Split Train-Test

In [7]:
train = raw_df[raw_df['SALE_DATE'] < '2019-04-01']
test = raw_df[raw_df['SALE_DATE'] >= '2019-04-01']
train.shape, test.shape

((2517, 15), (647, 15))

# EDA

## PAIRWISE ANALYSIS


### `GROSS_SQUARE_FEET` vs. `SALE_PRICE` AND `LAND_SQUARE_FEET` vs. `SALE_PRICE`

In [0]:
px.scatter(train, x='GROSS_SQUARE_FEET', y='SALE_PRICE', trendline='lowess')

Notice how `SALE_PRICE` is greater than $0 for `GROSS_SQUARE_FEET == 0`, so we can drop them as they might represent data entry error or placeholder for NaN values.

In [0]:
(train['GROSS_SQUARE_FEET'] == 0).value_counts()

There are about 31 values that have `0` Gross Square Feet. Let's check if their `LAND_SQUARE_FEET` is 0 as well.

In [0]:
(train['LAND_SQUARE_FEET'] == 0).value_counts()

Let's plot them to see it...

In [0]:
px.scatter(train, x='LAND_SQUARE_FEET', y='SALE_PRICE', trendline='lowess')

Indeed there are 0s for `LAND_SQUARE_FEET` as well, let's see if the rows are the 0 for `GROSS_SQUARE_FEET` as well.

In [0]:
train.query('GROSS_SQUARE_FEET == 0 & LAND_SQUARE_FEET == 0')

Some of the commonalities are:

- Both have the same neighborhood - `BREEZY POINT` and `THROGS NECK`
- Both have the same block - `2, 4`
- Both have the same `BUILDING_CLASS_AT_PRESENT` and `BUILDING_CLASS_AT_TIME_OF_SALE` - `A8`

In [0]:
# Let's drop them 
train = train[train['GROSS_SQUARE_FEET'] != 0]
test = test[test['GROSS_SQUARE_FEET'] != 0]
train.shape, test.shape

### `BOROUGH` vs `PRICE`

In [0]:
train.groupby('BOROUGH')['SALE_PRICE'].describe()

We can see the mean and median sale prices are all different for the boroughs, let's plot them to visualize their difference. 

In [0]:
sns.catplot(x='BOROUGH', y='SALE_PRICE', data=train, kind='bar', color='grey', height=6, aspect=1.5);

### `NEIGHBORHOOD` vs. `SALE_PRICE`

In [0]:
train['NEIGHBORHOOD'].value_counts()

It looks like their is high cardinality in `NEIGHBORHOOD`, we can reduce cardinality by keeping only top 10, and rest grouped as `OTHER`.

In [0]:
# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# Filter locations based on top10 neighborhoods, and not as OTHERS

train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

sns.catplot(x='NEIGHBORHOOD', y='SALE_PRICE', data=train, color='grey',  kind='bar', height=6, aspect=2);
plt.xticks(rotation=45);

In [0]:
train['NEIGHBORHOOD'].value_counts()

### `BUILDING_CLASS` vs. `SALE_PRICE`

In [0]:
sns.catplot(x='BUILDING_CLASS_AT_PRESENT', y='SALE_PRICE', data=train, kind='box', color='grey', height=6, aspect=1.5);

In [0]:
sns.catplot(x='BUILDING_CLASS_AT_TIME_OF_SALE', y='SALE_PRICE', data=train, color='grey', kind='box', height=6, aspect=1.5);

In [0]:
train.groupby('BUILDING_CLASS_AT_TIME_OF_SALE')['SALE_PRICE'].describe()

It would be appropriate to choose `BUILDING_CLASS_AT_TIME_OF_SALE` since, it would the most recent `BUILDING_CLASS` related to `SALE_PRICE`.

Another thing to note is `SALE_PRICE` of:

- `A9`, `A1`, `A5`, `A0`, `A2` are almost similar, mean, and median values compared to `A3`, `A4`, `A6`, `S1` and `S0`.

In [0]:
# Let's group A0, A1, A2, A5, A9 in one group and separate A3, A4, A6, S1 and S0
# TODO

In [0]:
train['BUILDING_CLASS_AT_TIME_OF_SALE'].value_counts()

### `ZIP_CODE` vs. `SALE_PRICE`

In [0]:
sns.lmplot(x='ZIP_CODE', y='SALE_PRICE', data=train, height=6, aspect=1.5, scatter_kws=dict(alpha=0.05));

It looks like we can bin the zipcodes.

In [0]:
train['ZIPCODE_CAT'] = pd.cut(train['ZIP_CODE'], [0, 10400, 10480, 11200, 11250, 11400, 11500, 11700], right=True, labels=['ZIP_10400', 'ZIP_10480', 'ZIP_11200', 'ZIP_11250', 'ZIP_11400', 'ZIP_11500', 'ZIP_11700'])
test['ZIPCODE_CAT'] = pd.cut(test['ZIP_CODE'], [0, 10400, 10480, 11200, 11250, 11400, 11500, 11700], right=True, labels=['ZIP_10400', 'ZIP_10480', 'ZIP_11200', 'ZIP_11250', 'ZIP_11400', 'ZIP_11500', 'ZIP_11700'])
sns.catplot(x='ZIPCODE_CAT', y='SALE_PRICE', data=train, height=6, aspect=1.5, color='grey')
plt.xticks(rotation=45);

After binning the zipcodes, it looks like the prices are now different with regards to their bins. 

In [0]:
#### DON'T FORGET TO DROP ZIP_CODE ####
train['ZIPCODE_CAT'] = train['ZIPCODE_CAT'].astype(object)
test['ZIPCODE_CAT'] = test['ZIPCODE_CAT'].astype(object)
train = train.drop('ZIP_CODE', axis=1)
test = test.drop('ZIP_CODE', axis=1)

### `YEAR_BUILT` vs. `SALE_PRICE`

In [0]:
train['YEAR_BUILT'].value_counts()

Since, all the values are the same we can drop the column.

In [0]:
train = train.drop('YEAR_BUILT', axis=1)
test = test.drop('YEAR_BUILT', axis=1)

### `BLOCK` vs. `SALE_PRICE`

`BLOCK:` 

> A Tax Block is a sub-division of the borough on which real properties are located. The  Department  of  Finance  uses  a  Borough-Block-Lot  classification  to  label  all  real  property  in  the  City.

In [0]:
train['BLOCK'].describe()

In [0]:
sns.lmplot(x='BLOCK', y='SALE_PRICE', data=train, height=6, aspect=1.5, scatter_kws=dict(alpha=0.05));

It doesn't look like `BLOCK` is a good predictor of `SALE_PRICE`.

### `LOT` vs. `SALE_PRICE`

`LOT:`

> A Tax Lot is a subdivision of a Tax Block and represents the property unique location.

In [0]:
sns.lmplot(x='LOT', y='SALE_PRICE', data=train, height=6, aspect=1.5, scatter_kws=dict(alpha=0.05));

Again, both `BLOCK` and `LOT` are not good predictors of `SALE_PRICE`.

### `TOTAL_UNITS, RESIDENTIAL_UNITS, COMMERCIAL_UNITS vs. SALE_PRICE`

First, let's verify if the `RESIDENTIAL_UNITS` + `COMMERICAL_UNITS` = `TOTAL_UNITS`

if it is, we can just discard other, and keep `TOTAL_UNITS`

In [0]:
(train['RESIDENTIAL_UNITS'] + train['COMMERCIAL_UNITS']).value_counts()

In [0]:
train['TOTAL_UNITS'].value_counts()

Indeed, it is the same, let's keep `TOTAL_UNITS`, and drop the others out.

In [0]:
train = train.drop(['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS'], axis=1)
test = test.drop(['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS'], axis=1)

In [0]:
sns.catplot(x='TOTAL_UNITS', y='SALE_PRICE', data=train, kind='bar', color='grey', height=6, aspect=1.5);

We can see `TOTAL_UNITS` is good indicator of `SALE_PRICE`. 

# ONE-HOT ENCODINGS

Let's try out one-hot encodings of chosen cateogrical features:

- `TOTAL_UNITS` 
- `ZIPCODE_CAT`
- `BOROUGH`
- `NEIGHBORHOOD`
- `BUILDING_CLASS_AT_TIME_OF_SALE`


In [0]:
train.describe(exclude='number').T.sort_values(by='unique')

In [0]:
encoder = ce.OneHotEncoder(use_cat_names=True)
cat_features = ['TOTAL_UNITS', 'BOROUGH', 'ZIPCODE_CAT', 'NEIGHBORHOOD', 'BUILDING_CLASS_AT_TIME_OF_SALE']
numeric_features = ['GROSS_SQUARE_FEET', 'LAND_SQUARE_FEET', 'SALE_PRICE']
combined_features = cat_features + numeric_features
train_encoded = encoder.fit_transform(train[combined_features])
test_encoded = encoder.transform(test[combined_features])
train_encoded.head()

So, encodings work great, there was no issue with it, let's check correlations.

# CORRELATIONS

Let's summarize what we know so far:

- Good Predictors:
  - `TOTAL_UNITS`
  - `GROSS_SQUARE_FEET` and `LAND_SQUARE_FEET`
  - `BOROUGH`
  - `NEIGHBORHOOD`
  - `ZIPCODE_CAT`
  - May be `BUILDING_CLASS_AT_TIME_OF_SALE` if we bin it.


## Correlation Heatmap

In [0]:
correlation_heatmap(train_encoded, figsize=(25, 25))

### Important Correlations

In [0]:
correlations(train, 'SALE_PRICE', ['GROSS_SQUARE_FEET', 'LAND_SQUARE_FEET', 'TOTAL_UNITS'])

# Linear Regression Model

In [0]:
def linear_model_results(data=None, features=None, target=None, one_hot=False, card_n=12, scaling=False, kbest=False, k_val=10, by_date=False, date_col=None, cutoff=None, seed=12345):
    if data is None:
        raise ValueError( "The parameter 'data' must be assigned a non-nil reference to a Pandas DataFrame")
    if (features is None) or (not isinstance(features, list)):
        raise ValueError( "The parameter 'features' must be a non-nil reference and list data type")
    if (target is None) or (not isinstance(target, str)):
        raise ValueError( "The parameter 'target' must be a non-nil reference and string object")
    
    # Import to split the data into training/testing sets
    from sklearn.model_selection import train_test_split
    # Import Linear Model
    from sklearn.linear_model import LinearRegression
    # Import metrics
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    # Extract features 
    if by_date and cutoff and date_col:
        cutoff = pd.to_datetime(cutoff)
        train = data[data[date_col] < cutoff]
        test = data[data[date_col] >= cutoff]
        X_train = train[features]
        y_train = train[target]
        X_test = test[features]
        y_test = test[target]
    else:
        # Train-Test Split
        X = df[features]
        y = df[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=seed)
    
    # One-hot encoding only low card. features
    if one_hot:
        # Check if features have low cardinality otherwise raise error
        if any(X_train.describe(exclude='number').T['unique'] > 12):
            raise ValueError(f'Cannot One-Hot encode, cardinality of one of the feature is greater than set cutoff {card_n}')
        # Import categorical encoder
        import category_encoders as ce
        # Do one-hot encoding
        encoder = ce.OneHotEncoder(use_cat_names=True)
        X_train = encoder.fit_transform(X_train)
        X_test = encoder.transform(X_test)
        # Assert columns are same
        assert X_train.shape[1] == X_test.shape[1]

    # Standard Scaling features
    if scaling:
        # Import standard scaler
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        # Assert columns are same
        assert X_train.shape[1] == X_test.shape[1]

    # Select K Best features
    if kbest and k:
        # Select the 10 features that best correlate with the target
        from sklearn.feature_selection import f_regression, SelectKBest
        selector = SelectKBest(score_func=f_regression, k=10)
        X_train = selector.fit_transform(X_train, y_train)
        X_test = selector.transform(X_test)
        assert X_train.shape[1] == X_test.shape[1]
    
    # Assign empty dictionary to hold results
    results = {}
    results['n'] = data.shape[0]

    # Instantiate Linear Regression Mode
    model = LinearRegression().fit(X_train, y_train)
    # Add model to results
    results['model'] = model
    # Add coefficients to results
    results['coefficients'] = model.coef_[0]

    # Calculate metrics on training data
    # Add R^2 
    results['train_r_squared'] = model.score(X_train, y_train)
    # Predict
    y_train_hat = model.predict(X_train)
    # MSE
    results['train_MSE'] = mean_squared_error(y_train, y_train_hat)
    # RMSE
    results['train_RMSE'] = np.sqrt(results['train_MSE'])
    # MAE
    results['train_MAE'] = mean_absolute_error(y_train, y_train_hat)

    # Calculate metrics on the test data
    # Add R^2 
    results['test_r_squared'] = model.score(X_test, y_test)
    # Predict
    y_test_hat = model.predict(X_test)
    # MSE
    results['test_MSE'] = mean_squared_error(y_test, y_test_hat)
    # RMSE
    results['test_RMSE'] = np.sqrt(results['test_MSE'])
    # MAE
    results['test_MAE'] = mean_absolute_error(y_test, y_test_hat)
    return results


def lmodel(X_train, X_test, y_train, y_test):
    # Instantiate Linear Regression Mode
    model = LinearRegression().fit(X_train, y_train)

    # Store metrics   
    results = {}
    # Add model to results
    results['model'] = model
    # Add coefficients to results
    results['coefficients'] = model.coef_[0]
    # Calculate metrics on training data
    # Add R^2 
    results['train_r_squared'] = model.score(X_train, y_train)
    # Predict
    y_train_hat = model.predict(X_train)
    # MSE
    results['train_MSE'] = mean_squared_error(y_train, y_train_hat)
    # RMSE
    results['train_RMSE'] = np.sqrt(results['train_MSE'])
    # MAE
    results['train_MAE'] = mean_absolute_error(y_train, y_train_hat)

    # Calculate metrics on the test data
    # Add R^2 
    results['test_r_squared'] = model.score(X_test, y_test)
    # Predict
    y_test_hat = model.predict(X_test)
    # MSE
    results['test_MSE'] = mean_squared_error(y_test, y_test_hat)
    # RMSE
    results['test_RMSE'] = np.sqrt(results['test_MSE'])
    # MAE
    results['test_MAE'] = mean_absolute_error(y_test, y_test_hat)
    return results


def print_lm(results):
    print("""
    ----------- Linear Regression Model Results -------------
    Training Set
    R^2: {:.2f} (Explained variance score: 1 is perfect prediction)
    MSE: {:.2f}
    RMSE: {:.2f}
    MAE: ${:.2f}

    Test Set
    R^2: {:.2f} (Explained variance score: 1 is perfect prediction)
    MSE: {:.2f}
    RMSE: {:.2f}
    MAE: ${:.2f}
    """.format(results['train_r_squared'], results['train_MSE'], results['train_RMSE'], results['train_MAE'], results['test_r_squared'], results['test_MSE'], results['test_RMSE'], results['test_MAE']))

### Simple Baseline Model

In [0]:
# 1. Extract features
target = 'SALE_PRICE'
features = ['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET']

# 2. Instantiate Linear Regression Model
results = linear_model_results(raw_df, features, target, by_date=True, date_col='SALE_DATE', cutoff='2019-04-01')

print_lm(results)

### Complex Linear Regression Model

#### 1. Model with One-Hot Encoding

In [0]:
## Subset features, and select target
cat_features = ['TOTAL_UNITS', 'BOROUGH', 'ZIPCODE_CAT', 'NEIGHBORHOOD', 'BUILDING_CLASS_AT_TIME_OF_SALE']
numeric_features = ['GROSS_SQUARE_FEET', 'LAND_SQUARE_FEET', 'SALE_PRICE']
features = cat_features + numeric_features
target = 'SALE_PRICE'

## Split features to X, y - train, test split
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

## 1. Do one-hot encoding
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

# Assert columns are same
assert X_train_encoded.shape[1] == X_test_encoded.shape[1]

results = lmodel(X_train_encoded, X_test_encoded, y_train, y_test)
print_lm(results)

#### 2. Model with Standard Scaling

In [0]:
## 2. Standard Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Assert columns are same
assert X_train_scaled.shape[1] == X_test_scaled.shape[1]

results = lmodel(X_train_scaled, X_test_scaled, y_train, y_test)
print_lm(results)

### 3. Model with Select K Best features

In [0]:
## 3. Select K Best features
# Select the 10 features that best correlate with the target
selector = SelectKBest(score_func=f_regression, k=10)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)
assert X_train_selected.shape[1] == X_test_selected.shape[1]

# Which features were selected?
all_names = X_train_selected.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

results = lmodel(X_train_selected, X_test_selected, y_train, y_test)
print_lm(results)